In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import numpy as np
import time

(train_data, val_data), meta = tfds.load('cats_vs_dogs', split=['train[:80%]', 'train[80%]'], 
                                                        with_info=True, as_supervised=True)

num_train=meta.features['train'].num_examples

c:\ProgramData\Anaconda3\envs\data_sci\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

KeyboardInterrupt

